# prefixtune: default program

In [1]:
from prefixtune import *
import os, sys

/Users/gouttham/NLP_commit/nlp3/prefixtune/env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/gouttham/NLP_commit/nlp3/prefixtune/env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Run the default solution on small

In [2]:
basemodel = 'distilgpt2'
table_to_text = TableToText("peft", basemodel=basemodel)
model = AutoModelForCausalLM.from_pretrained(basemodel)
decoder_output = table_to_text.decode(model, '../data/input/small.txt')
print("\n".join(decoder_output))

10it [00:13,  1.39s/it]

0||  __________________________________________________________________________   It's been a busy day for me this week, and I've been working hard to make sure you're on the right track. I'm going to take a look at some of the most popular ways to make your
1||  _______________   I’ve been waiting for the last few days for the release of the latest version of Windows Phone 8.1.0.1, but I’m not sure what it means to be a Windows Phone user
2||  __________________________________________   The U.S. Department of Justice (DOJ) announced today that the Justice Department has filed a lawsuit against the Federal Bureau of Investigation (FBI) for the unlawful use of a private email server in the United States
3||  _______________________________________________________________________________   This is the first in a series of blog posts on the subject of a new book, “The Rise and Fall of the American Civil War.” The following is an excerpt from the book The Rise and
4||  _______________   T

Ignore the warnings from the transformers library. They are expected to occur.

## Evaluate the default output

See `bleu.py`

## Documentation

Write some beautiful documentation of your program here.

Try 1: epochs 3; prefixprojection : False
dev.out score: 0.9345
small.out score: 1.1543

Try 2: epochs 3; prefixprojection : True
dev.out score: 16.2812
small.out score: 16.8922

Try 3: epochs 3; prefixprojection : True; no_repeat_ngram_size = 3 (Got it from wwa118)
small.out score: 22.1565
dev.out score: 31.1741

Try 4: epochs 3; prefixprojection : True; no_repeat_ngram_size = 3 (Got it from wwa118); num_sequences=5; selection stratergy based on score
small.out score: 22.7204
dev.out score: 29.9943

Try 5: epochs 3; prefixprojection : True; no_repeat_ngram_size = 3 (Got it from wwa118); num_sequences=5; selection stratergy based on length of the text
small.out score: 20.8834
dev.out score: 30.5553

## Analysis

Do some analysis of the results. What ideas did you try? What worked and what did not?

In [122]:
tokenizer = AutoTokenizer.from_pretrained(basemodel)

In [123]:
prompt = 'Convert the following table into English text: '
src = 'name : Alimentum | area : city centre | family friendly : no'

In [129]:
input_text = prompt + src + ' ' + tokenizer.bos_token + ' '
inputs = tokenizer(prompt + src + ' ' + tokenizer.bos_token + ' ', return_tensors="pt")
prediction = None

num_sequences = 5
with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=50,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        num_beams=5,
        top_p=0.9,
        temperature=1.0,
        num_return_sequences=num_sequences,
        no_repeat_ngram_size = 3,
        output_scores=True,
        return_dict_in_generate=True
    )
    # TODO you may want to generate more than one sequence and choose the best one!
    # text = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)

In [130]:
sequences_scores = outputs.sequences_scores
print(sequences_scores)

tensor([-1.4093, -1.4414, -1.4428, -1.4512, -1.4558])


In [132]:
text = tokenizer.batch_decode([outputs.sequences[torch.argmax(sequences_scores).item()]], skip_special_tokens=True)
text = text[0]
input_text = input_text.replace(tokenizer.bos_token,"")
generated_text = text[len(input_text):] if text.startswith(input_text) else text
generated_text = generated_text.lstrip().replace(prompt + src, "").replace("\n", " ")
generated_text = generated_text.strip().replace("\n", " ")

In [134]:
text.startswith(input_text)

True

selecting based on lenght of the text.

In [136]:
tokenizer = AutoTokenizer.from_pretrained(basemodel)

In [137]:
prompt = 'Convert the following table into English text: '
src = 'name : Alimentum | area : city centre | family friendly : no'

In [144]:
input_text = prompt + src + ' ' + tokenizer.bos_token + ' '
inputs = tokenizer(prompt + src + ' ' + tokenizer.bos_token + ' ', return_tensors="pt")
prediction = None

num_sequences = 5
with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=50,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        num_beams=5,
        top_p=0.9,
        temperature=1.0,
        num_return_sequences=num_sequences,
        no_repeat_ngram_size = 3,
        output_scores=True,
        return_dict_in_generate=True
    )
    # TODO you may want to generate more than one sequence and choose the best one!
    # text = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)

In [145]:
text = tokenizer.batch_decode(outputs.sequences, skip_special_tokens=True)
text = max(text, key=lambda i: len(i.split(' ')))
input_text = input_text.replace(tokenizer.bos_token,"")
generated_text = text[len(input_text):] if text.startswith(input_text) else text
generated_text = generated_text.lstrip().replace(prompt + src, "").replace("\n", " ")
generated_text = generated_text.strip().replace("\n", " ")

In [146]:
generated_text

'___________________________________________________________________________   We’ve been around for a while now, and I’m sure it’s a good time to give you a look at some of the most important and most important things to know about the internet'